In [53]:
# pip install pandas
# pip install numpy
# pip install scikit-learn
# pip install nltk
# pip install tensorflow
# pip install setuptools
# pip install seaborn
# pip install transformers

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')
import nltk
print(nltk.data.path)

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import setuptools.dist
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

from tensorflow.keras import backend as K
K.clear_session()

['C:\\Users\\Robin/nltk_data', 'c:\\Users\\Robin\\Dropbox\\SIT\\Y1S2\\ML\\nlp_sentimentanalysis\\.venv\\nltk_data', 'c:\\Users\\Robin\\Dropbox\\SIT\\Y1S2\\ML\\nlp_sentimentanalysis\\.venv\\share\\nltk_data', 'c:\\Users\\Robin\\Dropbox\\SIT\\Y1S2\\ML\\nlp_sentimentanalysis\\.venv\\lib\\nltk_data', 'C:\\Users\\Robin\\AppData\\Roaming\\nltk_data', 'C:\\nltk_data', 'D:\\nltk_data', 'E:\\nltk_data']


In [55]:
hate_df = pd.read_csv('./data_cleaned.csv')
hate_df.head()

,count,hate_speech_count,offensive_language_count,neither_count,class,tweet,binary_class,tweet_length,word_count
0,3,0,0,3,2,rt mayasolov woman shouldnt complain clean hou...,safe,140,25
1,3,0,3,0,1,rt mleew boy dat coldtyga dwn bad cuffin dat h...,unsafe,85,16
2,3,0,3,0,1,rt urkindofbrand dawg rt sbabylif ever fuck bi...,unsafe,120,21
3,3,0,2,1,1,rt cganderson vivabas look like tranni,unsafe,62,9
4,6,0,6,0,1,rt shenikarobert shit hear might true might fa...,unsafe,137,26


In [56]:
hate_df = hate_df[['tweet', 'class']]
hate_df.head()

,tweet,class
0,rt mayasolov woman shouldnt complain clean hou...,2
1,rt mleew boy dat coldtyga dwn bad cuffin dat h...,1
2,rt urkindofbrand dawg rt sbabylif ever fuck bi...,1
3,rt cganderson vivabas look like tranni,1
4,rt shenikarobert shit hear might true might fa...,1


In [57]:
hate_df.columns

Index(['tweet', 'class'], dtype='object')

In [58]:
#RENAMING THE COLUMNS
hate_df.rename(columns = {'tweet': 'text', 'class': 'label'}, inplace = True)
hate_df.columns

Index(['text', 'label'], dtype='object')

In [59]:
hate_df.isnull().sum()

text     0
label    0
dtype: int64

In [60]:
hate_df.shape

(24783, 2)

In [61]:
hate_df['label'].value_counts()

label
1    19190
2     4163
0     1430
Name: count, dtype: int64

In [62]:
hate_df.head()

,text,label
0,rt mayasolov woman shouldnt complain clean hou...,2
1,rt mleew boy dat coldtyga dwn bad cuffin dat h...,1
2,rt urkindofbrand dawg rt sbabylif ever fuck bi...,1
3,rt cganderson vivabas look like tranni,1
4,rt shenikarobert shit hear might true might fa...,1


In [63]:
hate_df.reset_index(drop = True, inplace = True)
hate_df.head()

,text,label
0,rt mayasolov woman shouldnt complain clean hou...,2
1,rt mleew boy dat coldtyga dwn bad cuffin dat h...,1
2,rt urkindofbrand dawg rt sbabylif ever fuck bi...,1
3,rt cganderson vivabas look like tranni,1
4,rt shenikarobert shit hear might true might fa...,1


Label Encoding (Converting categorical data to numerical inputs)

In [64]:
label_encoder = LabelEncoder()

# Stopwords Removal
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
word_tokenize("Hello world!")
# nltk.download('stopwords', download_dir="./")
# nltk.download('punkt', download_dir="./")

# LOADING THE STOPWORDS
# nltk.data.path.append(" ./nltk_data/corpora/stopwords/")
stop_words = set(stopwords.words('english'))

len(stop_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Robin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Robin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Robin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


198

In [65]:
import string

def stopword_removal(text):
    text = text.lower()
    all_words = nltk.word_tokenize(text)
    filtered_words = [
        word for word in all_words 
        if (word not in stop_words) and (word not in string.punctuation)
        ]
    return ' '.join(filtered_words)

In [67]:
hate_df['text'] = hate_df['text'].astype(str)

hate_df['text'] = hate_df['text'].apply(stopword_removal)

Tokenization and Padding (Tokenization replaces sensitive data, Padding to preserve size and fit data correctly)

In [69]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(hate_df['text'])

In [70]:
hate_seq = tokenizer.texts_to_sequences(hate_df['text'])
max_length = 50
hate_padded = pad_sequences(hate_seq, maxlen = max_length, padding = 'post')

In [71]:
#GENERATING LABELS IN NUMPY ARRAY FORMAT
hate_label = np.array(hate_df['label'])
print(hate_label.dtype)

int64


Model

In [72]:
# PREPARE SEPERATE INPUTS FOR EACH DATASET
hate_input = hate_padded

In [73]:
#DEFINING MULTIPLE INPUT LAYERS
hate_input_layer = keras.layers.Input(shape = (max_length,), name = 'hate_input')

In [74]:
#SHARED EMBEDDING LAYER
embedding_layer = keras.layers.Embedding(input_dim = len(tokenizer.word_index)+1, output_dim = 128)

#APPLY THE EMBEDDING LAYER TO EACH INPUT
hate_embedding = embedding_layer(hate_input_layer)

In [75]:
#SHARED LSTM LAYER
shared_lstm = keras.layers.LSTM(64, return_sequences = True)

#APPLY THE LSTM LAYER TO EACH INPUT
hate_lstm = shared_lstm(hate_embedding)

In [76]:
#SHARED GLOBAL AVERAGE LAYER AN DROPOUT LAYER
shared_pooling = keras.layers.GlobalAveragePooling1D()
shared_dropout = keras.layers.Dropout(0.5)

hate_features = shared_dropout(shared_pooling(hate_lstm))

In [77]:
len(hate_df['label'].unique())

3

In [78]:
#OUTPUT LAYERS
hate_output = keras.layers.Dense(3, activation = 'softmax', name = 'hate_output')(hate_features)

In [79]:
#COMPILATION
model = keras.models.Model(inputs = hate_input_layer,
                           outputs = hate_output)

model.compile(optimizer = 'adam',
             loss = {
                 'hate_output' : 'sparse_categorical_crossentropy'
             },
             metrics = {
                 'hate_output' : 'accuracy'
             })

In [80]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ hate_input (InputLayer)         │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 50, 128)        │     3,682,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 50, 64)         │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 64)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hate_output (Dense)             │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,732,035 (14.24 MB)

 Trainable params: 3,732,035 (14.24 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Takes roughly 10 mins to run 
print(hate_input.dtype)       # if it's a NumPy array

label_encoder.fit(hate_label)             # learn the mapping from label strings to integers
hate_label_encoded = label_encoder.transform(hate_label)
print(hate_label_encoded.dtype)  

#TRAINING THE MODEL WITH SEPERATE INPUTS
model.fit(hate_input,
          hate_label_encoded,
          epochs = 5,
          batch_size = 4)

int32
int64
Epoch 1/5


6196/6196 ━━━━━━━━━━━━━━━━━━━━ 116s 19ms/step - accuracy: 0.8325 - loss: 0.4806
Epoch 2/5
6196/6196 ━━━━━━━━━━━━━━━━━━━━ 115s 19ms/step - accuracy: 0.9249 - loss: 0.2260
Epoch 3/5
6196/6196 ━━━━━━━━━━━━━━━━━━━━ 119s 19ms/step - accuracy: 0.9599 - loss: 0.1222
Epoch 4/5
6196/6196 ━━━━━━━━━━━━━━━━━━━━ 118s 19ms/step - accuracy: 0.9769 - loss: 0.0737
Epoch 5/5
6196/6196 ━━━━━━━━━━━━━━━━━━━━ 115s 19ms/step - accuracy: 0.9880 - loss: 0.0396


In [32]:
prediction = model.predict({
                           'hate_input':hate_input})

775/775 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


In [33]:
prediction

array([[2.68708004e-06, 3.44257137e-06, 9.99993801e-01],
       [1.05336476e-04, 9.99885798e-01, 8.82382028e-06],
       [4.91086394e-05, 9.99949336e-01, 1.52868756e-06],
       ...,
       [3.07992959e-05, 9.99966502e-01, 2.66075722e-06],
       [9.42971201e-06, 9.99990225e-01, 3.23511614e-07],
       [1.64319863e-05, 6.98178883e-06, 9.99976516e-01]], dtype=float32)

In [34]:
hate_pred = np.argmax(prediction, axis = 1)

Manual Testing

In [ ]:
def classify_text(input_text):
    #PREPROCESS THE INPUT TEXT
    input_text_cleaned = stopword_removal(input_text)
    input_sequence = tokenizer.texts_to_sequences([input_text_cleaned])
    input_padded = pad_sequences(input_sequence, maxlen = max_length, padding = 'post')

    #PREDICTION
    predictions = model.predict(input_padded)
    hate_pred = np.argmax(predictions, axis=1)[0]

    hate_labels_text = ['offensive speech', 'Neither', 'Hate Speech']

    sub_label = hate_labels_text[hate_pred]

    return sub_label


In [88]:
input_text = 'devilgrimz vigxrart your fuck gay blacklist hoe hold tehgodclan anyway'
sub_label = classify_text(input_text)
print(f'Sub Label: {sub_label}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Sub Label: offensive speech
